In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from joblib import load
from sklearn.cluster import KMeans


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.10/site-packages/ipykernel

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [ ]:
!pip uninstall numpy

Found existing installation: numpy 2.2.4
Uninstalling numpy-2.2.4:
  Would remove:
    /opt/conda/bin/f2py
    /opt/conda/bin/numpy-config
    /opt/conda/lib/python3.10/site-packages/numpy-2.2.4.dist-info/*
    /opt/conda/lib/python3.10/site-packages/numpy.libs/libgfortran-040039e1-0352e75f.so.5.0.0
    /opt/conda/lib/python3.10/site-packages/numpy.libs/libquadmath-96973f99-934c22de.so.0.0.0
    /opt/conda/lib/python3.10/site-packages/numpy.libs/libscipy_openblas64_-6bb31eeb.so
    /opt/conda/lib/python3.10/site-packages/numpy/*
Proceed (Y/n)? 

In [1]:
#参数列表
kmeans_loaded = load('../model/kmeans/kmeans_40_双手合并.joblib') #分类文件夹根目录
all_path = '../SLR_dataset/seq_txt/'
save_path = '../SLR_dataset/kmeans_40_seq_双手合并_多头/'
all_folder = [i for i in os.listdir(all_path)]
quchong = False #去重

NameError: name 'load' is not defined

In [113]:
# 检查路径是否存在，不存在则创建（包括父目录）
if not os.path.exists(save_path):
    os.makedirs(save_path)
    print(f"目录已创建：{save_path}")
else:
    print(f"目录已存在：{save_path}")
def left_right_hand(data):
    left = data[0:63]
    right = data [63:]
    return left,right
def list_proLR(data):
    #该函数用于将读取到的数据变成左右手格式
    result = []
    for i in data:
        left,right = left_right_hand(i)
        result+=[left,right]
    return result

目录已存在：../SLR_dataset/kmeans_40_seq_双手合并_数据去重_三次滑动/


In [114]:
def remove_consecutive_duplicates(sequence):
    if not sequence:  # 空列表直接返回
        return []
    
    result = [sequence[0]]  # 初始化结果列表，包含第一个元素
    for num in sequence[1:]:  # 从第二个元素开始遍历
        if num != result[-1]:  # 当前元素与结果中最后一个元素不同
            result.append(num)
    return result

In [115]:
def predict_with_outlier(X, threshold=0.1):
    # distances = kmeans_loaded.transform(X)  # 计算到所有中心的距离
    # min_distances = np.min(distances, axis=1)
    
    # 分类结果（离群点设为0）
    labels = kmeans_loaded.predict(X)
    # labels[min_distances > threshold] = 999  # 阈值需根据数据分布调整
    
    return labels

In [116]:
def smart_clean(seq, max_repeats=1):
    cleaned = []
    i = 0
    n = len(seq)
    
    while i < n:
        best_pattern = None
        best_length = 0
        
        # 检测从当前位置开始的最长重复模式（包括单个元素的重复）
        for pattern_len in range(1, 5):  # 检测1到4长度的模式
            if i + pattern_len > n:
                continue
                
            pattern = seq[i:i+pattern_len]
            repeats = 1
            
            # 计算重复次数
            while i + (repeats + 1) * pattern_len <= n:
                next_segment = seq[i + repeats * pattern_len : i + (repeats + 1) * pattern_len]
                if next_segment == pattern:
                    repeats += 1
                else:
                    break
            
            # 如果重复次数超过阈值，且比之前找到的模式更长，则更新最佳模式
            if repeats > max_repeats and pattern_len * repeats > best_length:
                best_pattern = pattern
                best_length = pattern_len * repeats
        
        if best_pattern:
            # 保留1次模式（如果是单个元素重复，只保留1次）
            cleaned.extend(best_pattern)
            i += best_length
        else:
            cleaned.append(seq[i])
            i += 1
    return cleaned

In [110]:
maxlen = 0
for i in all_folder:
    all_new_labels = []
    folder = os.path.join(all_path,i)
    file_name = [file for file in os.listdir(folder) if file.endswith('.avi')]
    for j in file_name:
        file_path = os.path.join(folder,j)
        datalist = []
        with open(file_path, 'r') as file:  # 以读取模式打开文件
            for line in file:       # 逐行读取文件内容
                # 按空格分割每一行，并将每个部分转换为浮点型
                float_values = [float(x) for x in line.split()]
                datalist.append(float_values)  # 将转换后的浮点型列表添加到 datalist
        new_labels = predict_with_outlier(datalist)
        # print(len(new_labels)) #212
        # new_labels = [i for i in new_labels]
        # # print(new_labels)
        all_new_labels.append(new_labels)
    all_new_labels = [[x for x in j if x != 999] for j in all_new_labels]
    for cishu in range(3):
        all_new_labels = [smart_clean(seq, max_repeats=1) for seq in all_new_labels]
    # print(len(all_new_labels))
    for seq in all_new_labels:
        if len(seq) > maxlen:
            maxlen = len(seq)
    save_file_name = os.path.join(save_path,i)+'.npy'
    save_data = np.array(all_new_labels,dtype=object)
    # print(save_data)
    np.save(save_file_name,save_data)
    print(save_file_name)
        # all_new_labels.append(remove_consecutive_duplicates(new_labels))
print(maxlen)

../SLR_dataset/kmeans_40_seq_双手合并_数据去重_三次滑动/000000.npy
../SLR_dataset/kmeans_40_seq_双手合并_数据去重_三次滑动/000001.npy
../SLR_dataset/kmeans_40_seq_双手合并_数据去重_三次滑动/000002.npy
../SLR_dataset/kmeans_40_seq_双手合并_数据去重_三次滑动/000003.npy
../SLR_dataset/kmeans_40_seq_双手合并_数据去重_三次滑动/000004.npy
../SLR_dataset/kmeans_40_seq_双手合并_数据去重_三次滑动/000005.npy
../SLR_dataset/kmeans_40_seq_双手合并_数据去重_三次滑动/000006.npy
../SLR_dataset/kmeans_40_seq_双手合并_数据去重_三次滑动/000007.npy
../SLR_dataset/kmeans_40_seq_双手合并_数据去重_三次滑动/000008.npy
../SLR_dataset/kmeans_40_seq_双手合并_数据去重_三次滑动/000009.npy
../SLR_dataset/kmeans_40_seq_双手合并_数据去重_三次滑动/000010.npy
../SLR_dataset/kmeans_40_seq_双手合并_数据去重_三次滑动/000011.npy
../SLR_dataset/kmeans_40_seq_双手合并_数据去重_三次滑动/000012.npy
../SLR_dataset/kmeans_40_seq_双手合并_数据去重_三次滑动/000013.npy
../SLR_dataset/kmeans_40_seq_双手合并_数据去重_三次滑动/000014.npy
../SLR_dataset/kmeans_40_seq_双手合并_数据去重_三次滑动/000015.npy
../SLR_dataset/kmeans_40_seq_双手合并_数据去重_三次滑动/000016.npy
../SLR_dataset/kmeans_40_seq_双手合并_数据去重_三次滑动/000017.npy
../SLR_dat

In [106]:
# for i in all_new_labels:
#     print(i)

In [61]:
all_new_labels = [[x for x in j if x != 999] for j in all_new_labels]

In [81]:
all_new_labels = [smart_clean(seq, max_repeats=1) for seq in all_new_labels]

In [ ]:
all_new_labels

In [30]:
cleandata = []
for i in all_new_labels:
    cleandata.append(smart_clean(smart_clean(i)))

In [29]:
for i in all_new_labels[:5]:
    print(i)

[6, 33, 26, 15, 6, 2, 15, 6, 20, 38, 1, 4, 1, 4, 1, 4, 38, 4, 38, 4, 38, 4, 38, 4, 38, 4, 38, 20, 13, 38, 1, 4, 1, 4, 38, 4, 38, 20, 38, 4, 38, 1, 6, 1, 4, 1, 4, 10, 37, 27, 1, 20, 1, 38, 4, 38, 4, 38, 37, 38, 37, 10, 37, 10, 37, 10, 37, 6, 10, 37, 10, 37, 6]
[6, 33, 15, 33, 29, 33, 6, 38, 1, 4, 1, 38, 20, 38, 6, 20, 6, 38, 6, 10, 37, 38, 4, 38, 4, 38, 6, 38, 1, 6, 1, 10, 27, 10, 37, 1, 4, 1, 4, 1, 4, 10, 37, 1, 4, 1, 37, 1, 17, 10, 37, 10, 37, 10, 37, 10, 37, 10, 1, 37, 10, 20, 1, 10, 20, 1, 10, 1, 37, 10, 37, 1, 6, 1, 6]
[6, 33, 6, 2, 30, 25, 32, 25, 32, 8, 32, 10, 37, 1, 39, 28, 1, 24, 17, 36, 17, 0, 17, 0, 17, 0, 17, 0, 17, 2, 32, 2, 9, 2, 35, 5]
[6, 33, 6, 1, 6, 10, 37, 10, 37, 10, 6, 10, 0, 17, 1, 17, 1, 39, 28, 24, 1, 24, 17, 0, 17, 36, 17, 0, 17, 0, 17, 2, 32, 2, 9, 2, 35, 5]
[6, 33, 15, 2, 3, 2, 30, 25, 32, 8, 32, 10, 18, 10, 37, 1, 4, 39, 28, 24, 1, 24, 17, 0, 2, 8, 2, 35, 5]


In [31]:
for i in cleandata[:5]:
    print(i)

[6, 33, 26, 15, 6, 2, 15, 6, 20, 38, 1, 4, 38, 4, 38, 20, 13, 38, 1, 4, 1, 4, 38, 4, 38, 20, 38, 4, 38, 1, 6, 1, 4, 1, 4, 10, 37, 27, 1, 20, 1, 38, 4, 38, 4, 38, 37, 38, 37, 10, 37, 6, 10, 37, 10, 37, 6]
[6, 33, 15, 33, 29, 33, 6, 38, 1, 4, 1, 38, 20, 38, 6, 20, 6, 38, 6, 10, 37, 38, 4, 38, 4, 38, 6, 38, 1, 6, 1, 10, 27, 10, 37, 1, 4, 10, 37, 1, 4, 1, 37, 1, 17, 10, 37, 10, 1, 37, 10, 20, 1, 10, 20, 1, 10, 1, 37, 10, 37, 1, 6, 1, 6]
[6, 33, 6, 2, 30, 25, 32, 25, 32, 8, 32, 10, 37, 1, 39, 28, 1, 24, 17, 36, 17, 0, 17, 2, 32, 2, 9, 2, 35, 5]
[6, 33, 6, 1, 6, 10, 37, 10, 37, 10, 6, 10, 0, 17, 1, 17, 1, 39, 28, 24, 1, 24, 17, 0, 17, 36, 17, 0, 17, 0, 17, 2, 32, 2, 9, 2, 35, 5]
[6, 33, 15, 2, 3, 2, 30, 25, 32, 8, 32, 10, 18, 10, 37, 1, 4, 39, 28, 24, 1, 24, 17, 0, 2, 8, 2, 35, 5]


In [ ]:
for i in all_folder:
    all_new_labels = []
    folder = os.path.join(all_path,i)
    file_name = [file for file in os.listdir(folder) if file.endswith('.avi')]
    for j in file_name:
        file_path = os.path.join(folder,j)
        datalist = []
        with open(file_path, 'r') as file:  # 以读取模式打开文件
            for line in file:       # 逐行读取文件内容
                # 按空格分割每一行，并将每个部分转换为浮点型
                float_values = [float(x) for x in line.split()]
                datalist.append(float_values)  # 将转换后的浮点型列表添加到 datalist
        new_labels = kmeans_loaded.predict(datalist)
        all_new_labels.append(new_labels)
    save_file_name = os.path.join(save_path,i)+'.npy'
    save_data = np.array(all_new_labels,dtype=object)
    # print(save_data)
    np.save(save_file_name,save_data)
    print(save_file_name)

In [4]:
for i in all_folder:
    all_new_labels = []
    folder = os.path.join(all_path,i)
    file_name = [file for file in os.listdir(folder) if file.endswith('.avi')]
    for j in file_name:
        file_path = os.path.join(folder,j)
        datalist = []
        with open(file_path, 'r') as file:  # 以读取模式打开文件
            for line in file:       # 逐行读取文件内容
                # 按空格分割每一行，并将每个部分转换为浮点型
                float_values = [float(x) for x in line.split()]
                datalist.append(float_values)  # 将转换后的浮点型列表添加到 datalist
        new_labels = kmeans_loaded.predict(datalist)
        all_new_labels.append(new_labels)
    save_file_name = os.path.join(save_path,i)+'.npy'
    save_data = np.array(all_new_labels,dtype=object)
    # print(save_data)
    np.save(save_file_name,save_data)
    print(save_file_name)

../SLR_dataset/kmeans_300_seq_双手合并/000000.npy
../SLR_dataset/kmeans_300_seq_双手合并/000001.npy
../SLR_dataset/kmeans_300_seq_双手合并/000002.npy
../SLR_dataset/kmeans_300_seq_双手合并/000003.npy
../SLR_dataset/kmeans_300_seq_双手合并/000004.npy
../SLR_dataset/kmeans_300_seq_双手合并/000005.npy
../SLR_dataset/kmeans_300_seq_双手合并/000006.npy
../SLR_dataset/kmeans_300_seq_双手合并/000007.npy
../SLR_dataset/kmeans_300_seq_双手合并/000008.npy
../SLR_dataset/kmeans_300_seq_双手合并/000009.npy
../SLR_dataset/kmeans_300_seq_双手合并/000010.npy
../SLR_dataset/kmeans_300_seq_双手合并/000011.npy
../SLR_dataset/kmeans_300_seq_双手合并/000012.npy
../SLR_dataset/kmeans_300_seq_双手合并/000013.npy
../SLR_dataset/kmeans_300_seq_双手合并/000014.npy
../SLR_dataset/kmeans_300_seq_双手合并/000015.npy
../SLR_dataset/kmeans_300_seq_双手合并/000016.npy
../SLR_dataset/kmeans_300_seq_双手合并/000017.npy
../SLR_dataset/kmeans_300_seq_双手合并/000018.npy
../SLR_dataset/kmeans_300_seq_双手合并/000019.npy
../SLR_dataset/kmeans_300_seq_双手合并/000020.npy
../SLR_dataset/kmeans_300_seq_双手合并

In [21]:
xxx = np.load('../SLR_dataset/kmeans_100_seq_双手合并/000000.npy',allow_pickle=True)

FileNotFoundError: [Errno 2] No such file or directory: '../SLR_dataset/kmeans_100_seq_双手合并/000000.npy'